In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout, HTML, Button, VBox
from IPython.display import display, clear_output
from bidi.algorithm import get_display
import arabic_reshaper

# הגדרות פונט עבור עברית
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['axes.unicode_minus'] = False

#לצערי  תיקון טקסט עברי לגרפים
def fix_hebrew_text(text):
    reshaped_text = arabic_reshaper.reshape(text)
    fixed_text = get_display(reshaped_text)
    return fixed_text


In [2]:
def pension_simulator(
    management_fee_from_acc_perc,
    management_fee_from_deposit_perc,
    years_to_calculate,
    current_accumulation_start,
    monthly_deposit,
    annual_return_perc
):
    management_fee_from_acc = management_fee_from_acc_perc / 100
    management_fee_from_deposit = management_fee_from_deposit_perc / 100
    annual_return = annual_return_perc / 100
    annual_deposit = monthly_deposit * 12

    data = []
    current_accumulation = current_accumulation_start

    for year in range(1, years_to_calculate + 1):
        fees_from_deposit = annual_deposit * management_fee_from_deposit
        fees_from_accumulation = current_accumulation * management_fee_from_acc
        effective_accumulation_for_return = current_accumulation + annual_deposit - fees_from_deposit - fees_from_accumulation
        return_on_accumulation = effective_accumulation_for_return * annual_return
        end_of_year_accumulation = effective_accumulation_for_return + return_on_accumulation
        total_fees = fees_from_deposit + fees_from_accumulation  # <--- חדש

        data.append({
            'שנה': year,
            'צבירה בתחילת שנה': current_accumulation,
            'הפקדות שנתיות': annual_deposit,
            'דמי נ. מהפקדה': fees_from_deposit,
            'דמי נ. מהצבירה': fees_from_accumulation,
            'תשואה שנתית': return_on_accumulation,
            'סך דמי ניהול (ש"ח)': total_fees,
            'צבירה בסוף שנה': end_of_year_accumulation
        })

        current_accumulation = end_of_year_accumulation

    df = pd.DataFrame(data)
    clear_output(wait=True)

    print("--- טבלת תוצאות שנתית ---")
    display(df.style.format({
        'צבירה בתחילת שנה': "{:,.2f} ש\"ח".format,
        'הפקדות שנתיות': "{:,.2f} ש\"ח".format,
        'דמי נ. מהפקדה': "{:,.2f} ש\"ח".format,
        'דמי נ. מהצבירה': "{:,.2f} ש\"ח".format,
        'תשואה שנתית': "{:,.2f} ש\"ח".format,
        'סך דמי ניהול (ש"ח)': "{:,.2f} ש\"ח".format,
        'צבירה בסוף שנה': "{:,.2f} ש\"ח".format
    }))

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(df['שנה'], df['צבירה בסוף שנה'], marker='o', linestyle='-', color='skyblue')
    plt.title(fix_hebrew_text('התפתחות הצבירה לאורך השנים'), fontsize=14, fontweight='bold')
    plt.xlabel(fix_hebrew_text('שנה'), fontsize=12)
    plt.ylabel(fix_hebrew_text('צבירה בסוף שנה (ש"ח)'), fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.ticklabel_format(style='plain', axis='y')

    plt.subplot(1, 2, 2)
    plt.plot(df['שנה'], df['דמי נ. מהפקדה'].cumsum(), marker='x', linestyle='--', color='red', label=fix_hebrew_text('דמי ניהול מהפקדה (מצטבר)'))
    plt.plot(df['שנה'], df['דמי נ. מהצבירה'].cumsum(), marker='s', linestyle='--', color='green', label=fix_hebrew_text('דמי ניהול מהצבירה (מצטבר)'))
    plt.plot(df['שנה'], df['תשואה שנתית'].cumsum(), marker='^', linestyle='-', color='blue', linewidth=2, label=fix_hebrew_text('תשואה שנתית (מצטבר)'))
    plt.plot(df['שנה'], df['סך דמי ניהול (ש"ח)'].cumsum(), marker='D', linestyle='-', color='purple', linewidth=2, label=fix_hebrew_text('סך דמי ניהול (מצטבר)'))

    plt.title(fix_hebrew_text('דמי ניהול ותשואה מצטברים לאורך השנים'), fontsize=14, fontweight='bold')
    plt.xlabel(fix_hebrew_text('שנה'), fontsize=12)
    plt.ylabel(fix_hebrew_text('סכום מצטבר (ש"ח)'), fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.ticklabel_format(style='plain', axis='y')

    plt.tight_layout()
    plt.show()

    print("\nהערה: סימולטור זה אינו כולל אינפלציה או שינויים עתידיים בתקנות.")
    print("התשואה מחושבת על הצבירה לאחר ניכוי דמי ניהול, בתוספת ההפקדות.")


In [3]:
# סליידרים
style = {'description_width': 'initial'}
slider_layout = Layout(width='90%')

fee_acc_slider = FloatSlider(min=0.00, max=0.5, step=0.01, value=0.17, description='דמי ניהול מהצבירה (%):', style=style, layout=slider_layout)
fee_deposit_slider = FloatSlider(min=0.0, max=6.0, step=0.1, value=1.7, description='דמי ניהול מההפקדה (%):', style=style, layout=slider_layout)
years_slider = IntSlider(min=1, max=67, step=1, value=45, description='מספר שנים לחישוב:', style=style, layout=slider_layout)
current_acc_slider = FloatSlider(min=0, max=5000000, step=1000, value=197000, description='סכום צבירה נוכחי (ש"ח):', style=style, layout=slider_layout)
monthly_deposit_slider = FloatSlider(min=0, max=10000, step=100, value=3541, description='סכום הפקדה חודשי (ש"ח):', style=style, layout=slider_layout)
annual_return_slider = FloatSlider(min=-5.0, max=100, step=0.1, value=5.0, description='תשואה שנתית (%):', style=style, layout=slider_layout)

header_html = HTML("<h2>הזן פרמטרים לסימולטור הפנסיה:</h2>")
display(header_html)

interact(
    pension_simulator,
    management_fee_from_acc_perc=fee_acc_slider,
    management_fee_from_deposit_perc=fee_deposit_slider,
    years_to_calculate=years_slider,
    current_accumulation_start=current_acc_slider,
    monthly_deposit=monthly_deposit_slider,
    annual_return_perc=annual_return_slider
)

# --------------------------
# כפתור השוואת מסלולים
# --------------------------
comparison_data = pd.DataFrame()

compare_button = Button(description="השווה מסלול", button_style='info')

def on_compare_button_clicked(b):
    global comparison_data

    acc_fee = fee_acc_slider.value
    dep_fee = fee_deposit_slider.value
    years = years_slider.value
    current_acc = current_acc_slider.value
    monthly_dep = monthly_deposit_slider.value
    annual_return = annual_return_slider.value

    annual_deposit = monthly_dep * 12
    current = current_acc
    results = []

    for year in range(1, years + 1):
        f_dep = annual_deposit * (dep_fee / 100)
        f_acc = current * (acc_fee / 100)
        total_fees = f_dep + f_acc
        effective = current + annual_deposit - f_dep - f_acc
        profit = effective * (annual_return / 100)
        current = effective + profit

        results.append({
            'שנה': year,
            f'{acc_fee:.2f}%/{dep_fee:.2f}%': total_fees
        })

    new_df = pd.DataFrame(results).set_index('שנה')

    if comparison_data.empty:
        comparison_data = new_df
    else:
        comparison_data = comparison_data.join(new_df, how='outer')

    clear_output(wait=True)
    display(header_html)
    display(VBox([
        fee_acc_slider, fee_deposit_slider, years_slider,
        current_acc_slider, monthly_deposit_slider, annual_return_slider, compare_button
    ]))
    display(HTML("<h3>השוואת מסלולים לפי סך דמי ניהול:</h3>"))
    display(comparison_data.style.format("{:,.2f} ש\"ח".format))

compare_button.on_click(on_compare_button_clicked)
display(compare_button)


HTML(value='<h2>הזן פרמטרים לסימולטור הפנסיה:</h2>')

interactive(children=(FloatSlider(value=0.17, description='דמי ניהול מהצבירה (%):', layout=Layout(width='90%')…

Button(button_style='info', description='השווה מסלול', style=ButtonStyle())

In [4]:
comparison_data

""
